In [0]:
from pyspark.sql import functions as F

In [0]:
#read log content for date 20220401
df = (spark.read.json("dbfs:/mnt/simple_etl_adls2/raw/"))

In [0]:
#add column log date
df = (df.withColumn("source_path", F.input_file_name())
        .withColumn("file_date_str", F.regexp_extract("source_path", r'(\d{8})(?=\.json$)', 1))
        .withColumn("log_date", F.to_date("file_date_str", "yyyyMMdd"))
        .drop("file_date_str")
)

In [0]:
#select needed columns
df = df.select("_source.*", "log_date")


In [0]:
#refine column names
df = (df.withColumnRenamed("AppName", "app_name")
        .withColumnRenamed("Contract", "contract")
        .withColumnRenamed("Mac", "mac")
        .withColumnRenamed("TotalDuration", "total_duration")
)

In [0]:
(df.write.format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .save("/mnt/simple_etl_adls2/bronze"))
